# Model experiments

This notebook is for experimenting with the choice of model and performance variations between different time periods and clinics.

1. Train and evaluate current best model on total dataset (without differentiating between clinics)
2. Apply IECV to different models and check heteregeneity with respect to clinics
3. Train models per clinic and check performance
4. Use clinic as categorical feature in tree based models and check performance

## Load packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    StratifiedGroupKFold,
    TimeSeriesSplit,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, SplineTransformer

from noshow.features.feature_pipeline import create_features
from noshow.model.predict import create_prediction
from noshow.preprocessing.load_data import (
    load_appointment_csv,
    process_appointments,
    process_postal_codes,
)


## Load data and split in to train and test

In [ ]:
featuretable = pd.read_parquet("../data/processed/featuretable.parquet")

featuretable["no_show"] = (
    featuretable["no_show"].replace({"no_show": "1", "show": "0"}).astype(int)
)
featuretable["hour"] = featuretable["hour"].astype("category")
featuretable["weekday"] = featuretable["weekday"].astype("category")

print(featuretable.dtypes)

X, y = featuretable.drop(columns="no_show"), featuretable["no_show"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, shuffle=False
)
train_groups = X.index.get_level_values("pseudo_id")

In [ ]:
lgboost_model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=300,
    categorical_features=["hour", "weekday"],
)

categorical_features = ["hour", "weekday"]
continuous_features = X.columns.difference(categorical_features)

preprocessor = ColumnTransformer(
    transformers=[
        (
            "continuous",
            Pipeline([("scaler", RobustScaler()), ("spline", SplineTransformer())]),
            continuous_features,
        ),
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

log_model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(penalty=None)),
    ]
)

## Train and evaluate current best model on total dataset (without differentiating between clinics)

In [ ]:
def cv_auc_curve(X_train, y_train, model, cv, train_groups=None, title=None):
    fpr = {}
    tpr = {}
    roc_auc = {}
    test_indices = {}

    fig, ax = plt.subplots()

    for i, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train, train_groups)):
        X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_train_cv, y_train_cv)

        y_score = model.predict_proba(X_test_cv)[:, 1]
        fpr[i], tpr[i], _ = roc_curve(y_test_cv, y_score)
        roc_auc[i] = roc_auc_score(y_test_cv, y_score)
        test_indices[i] = test_idx

        ax.plot(fpr[i], tpr[i], c="b", alpha=0.15)

    ax.plot([0, 1], [0, 1], "k--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    # Add mean AUC and standard deviation to the legend
    mean_auc = np.mean(list(roc_auc.values()))
    std_auc = np.std(list(roc_auc.values()))
    ax.legend([f"ROC curve (AUC = {mean_auc:.3f} +/- {std_auc:.3f})"])
    if title:
        ax.set_title(title)
    fig.show()

    return roc_auc, test_indices

In [ ]:
_, _ = cv_auc_curve(
    X,
    y,
    lgboost_model,
    StratifiedGroupKFold(n_splits=5),
    train_groups,
)

In [ ]:
_, _ = cv_auc_curve(
    X,
    y,
    log_model,
    StratifiedGroupKFold(n_splits=5),
    train_groups,
)

## Check stratification

In [ ]:
# create modified version of create_treatment_groups
def create_treatment_groups(
    predictions: pd.DataFrame, patients: pd.DataFrame, bin_edges
) -> pd.DataFrame:
    """
    Create treatment groups based on predictions.

    Parameters
    ----------
    predictions : pd.DataFrame
        DataFrame containing predictions.
    patients : pd.DataFrame
        A dataframe containing the treatment groups of patients
    bin_edges : list
        List of edges defining the bins for prediction scores.

    Returns
    -------
    pd.DataFrame
        DataFrame with treatment group assignments.

    Raises
    ------
    ValueError
        If the predictions DataFrame is empty.
    """
    if predictions.empty:
        raise ValueError("The predictions DataFrame is empty.")

    # get unique patient ids
    unique_patient_ids = predictions["pseudo_id"].unique().tolist()
    relevant_patients = patients[patients["id"].isin(unique_patient_ids)]

    if not relevant_patients.empty:
        # Merge predictions with patients to get treatment group
        predictions = pd.merge(
            predictions,
            relevant_patients[["id", "treatment_group"]],
            right_on="id",
            left_on="pseudo_id",
            how="left",
        )
    else:
        predictions.loc[:, "treatment_group"] = None

    # Create prediction score bins using quantile bins
    predictions = predictions.sort_values(["prediction"], ascending=False)

    # only keep top prediction per patient for treatment/control split
    deduplicated = predictions.drop_duplicates(subset="pseudo_id", keep="first")
    # only keep patients without treatment group
    deduplicated = deduplicated[deduplicated["treatment_group"].isnull()]
    deduplicated["score_bin"] = pd.cut(deduplicated["prediction"], bins=bin_edges, include_lowest=True, labels=False, right=True)

    # Create stratified randomization in control and treatment groups using
    deduplicated = deduplicated.sort_values(["hoofdagenda", "prediction"])
    random.seed(1234)  # Set fixed seed for reproducibility
    deduplicated["treatment_group"] = deduplicated.groupby(
        ["hoofdagenda", "score_bin"],
        observed=True,
        # create random treatment group assignment
    )["prediction"].transform(lambda x: (np.arange(len(x)) + random.randint(0, 1)) % 2)

    # update predictions with treatment groups from deduplicated based on pseudo_id
    merged = predictions.merge(
        deduplicated[["pseudo_id", "treatment_group", "score_bin"]],
        on="pseudo_id",
        how="left",
        suffixes=("", "_new"),
    )
    # Update the treatment_group values where there's a new value
    merged["treatment_group"] = merged["treatment_group"].fillna(merged["treatment_group_new"])

    deduplicated['id'] = deduplicated['pseudo_id']
    # append deduplicated to patients
    patients = pd.concat([patients, deduplicated[["id", "treatment_group"]]], ignore_index=True).reset_index(drop=True)

    return merged, patients

In [ ]:
appointments_df = load_appointment_csv("../data/raw/poliafspraken_pilot.csv")
appointments_df = process_appointments(appointments_df).sort_index()
with open("../output/models/no_show_model_cv.pickle", "rb") as f:
    model = pickle.load(f)

all_postalcodes = process_postal_codes("../data/raw/NL.txt")
predictions_df = create_prediction(model, appointments_df, all_postalcodes)
# randomly assing hoofdagenda string to predictions_df
predictions_df["hoofdagenda"] = np.random.choice(["A", "B", "C"], len(predictions_df))
predictions_df = predictions_df.loc[
    predictions_df.index.get_level_values("start").year == 2022
]
predictions_df = predictions_df.reset_index()
predictions_df.shape

In [ ]:
# determine fixed quantiles:
# Calculate quantiles
n_bins = 4
quantiles = np.linspace(0, 1, n_bins + 1)
bin_edges = predictions_df['prediction'].quantile(quantiles).values
print(bin_edges)

In [ ]:

# get unique days in predictions_df["start"]
days = predictions_df["start"].dt.date.unique()
patients = pd.DataFrame(columns=["id", "treatment_group"])
final_predictions = []
for day in days:
    day_df = predictions_df[predictions_df["start"].dt.date == day]
    predict, patients = create_treatment_groups(day_df, patients, bin_edges)
    final_predictions.append(predict)
final_predictions= pd.concat(final_predictions)
print(len(patients)) == len(patients['id'].unique())
print(final_predictions.shape)
final_predictions.head()

In [ ]:
# get first entry for every pseuoo_id
first_app = final_predictions.sort_values("prediction", ascending=False).drop_duplicates(subset="pseudo_id", keep="first")

In [ ]:
fig, ax = plt.subplots()
first_app.groupby(["hoofdagenda", "treatment_group", "score_bin"]).size().unstack().plot(kind='bar', ax=ax)
plt.xlabel("hoofdagenda and score_bin")
plt.ylabel("Count")
plt.title("Distribution of Treatment Groups per hoofdagenda and score_bin")
plt.show()

In [ ]:
fig, ax = plt.subplots()
final_predictions.groupby(["hoofdagenda", "treatment_group", "score_bin"]).size().unstack().plot(kind='bar', ax=ax)
plt.xlabel("hoofdagenda and score_bin")
plt.ylabel("Count")
plt.title("Distribution of Treatment Groups per hoofdagenda and score_bin")
plt.show()


In [ ]:
print(patients['id'].nunique())
# find users with multiple treatment groups
patients[patients.duplicated(subset='id', keep=False)]

In [ ]:
print(len(patients['id'].unique()))

## Check temporal performance

In [ ]:
X_timesorted = X.sort_index(level="start")
y_timesorted = y.sort_index(level="start")

In [ ]:
roc_auc, test_indices = cv_auc_curve(
    X_timesorted,
    y_timesorted,
    lgboost_model,
    TimeSeriesSplit(n_splits=10),
)

In [ ]:
fold_times = [
    str(
        (
            X.iloc[idx].index.get_level_values("start").min().strftime("%Y-%m-%d"),
            X.iloc[idx].index.get_level_values("start").max().strftime("%Y-%m-%d"),
        )
    )
    for idx in test_indices.values()
]
fold_times

In [ ]:
roc_scores = pd.Series(roc_auc)
roc_scores.index = fold_times
roc_scores.plot.bar()

## Apply IECV

In [ ]:
appointments_df = load_appointment_csv("../data/raw/poliafspraken_no_show.csv")
appointments_df = process_appointments(appointments_df)
all_postalcodes = process_postal_codes("../data/raw/NL.txt")
appointments_features = create_features(
    appointments_df, all_postalcodes, minutes_early_cutoff=30
)

In [ ]:
appointments_features.columns

In [ ]:
appointments_features = (
    appointments_features[
        [
            "hoofdagenda",
            "hour",
            "weekday",
            "minutesDuration",
            "no_show",
            "prev_no_show",
            "prev_no_show_perc",
            "age",
            "dist_umcu",
            "prev_minutes_early",
            "earlier_appointments",
            "appointments_same_day",
            "appointments_last_days",
            "days_since_created",
            "days_since_last_appointment",
        ]
    ]
    .reset_index()
    .set_index(["pseudo_id", "start", "hoofdagenda"])
)

In [ ]:
appointments_features["no_show"] = (
    appointments_features["no_show"].replace({"no_show": 1, "show": 0}).astype(int)
)

appointments_features["hour"] = appointments_features["hour"].astype("category")
appointments_features["weekday"] = appointments_features["weekday"].astype("category")

X, y = appointments_features.drop(columns="no_show"), appointments_features["no_show"]

In [ ]:
train_groups = X.index.get_level_values("pseudo_id")

cv_auc_curve(
    X,
    y,
    lgboost_model,
    StratifiedGroupKFold(n_splits=5),
    train_groups,
)

In [ ]:
def group_leave_one_out(df):
    groups = df.index.get_level_values("hoofdagenda").unique()

    for test_group in groups:
        train_index = df.index.get_level_values("hoofdagenda") != test_group
        test_index = df.index.get_level_values("hoofdagenda") == test_group

        yield test_group, (np.where(train_index)[0], np.where(test_index)[0])

In [ ]:
def iecv_auc_curve(X_train, y_train, model):
    fpr = {}
    tpr = {}
    roc_auc = {}
    test_indices = {}

    fig, ax = plt.subplots()

    for group, (train_idx, test_idx) in group_leave_one_out(X_train):
        X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_train_cv, y_train_cv)

        y_score = model.predict_proba(X_test_cv)[:, 1]
        fpr[group], tpr[group], _ = roc_curve(y_test_cv, y_score)
        roc_auc[group] = roc_auc_score(y_test_cv, y_score)
        test_indices[group] = test_idx

        ax.plot(fpr[group], tpr[group], label=group)

    print(
        f"Mean AUC: {np.mean(list(roc_auc.values()))}"
        f"(+/- {np.std(list(roc_auc.values()))})"
    )
    ax.plot([0, 1], [0, 1], "k--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.legend()
    fig.show()

    return roc_auc, test_indices

In [ ]:
iecv_auc_curve(X, y, lgboost_model)

In [ ]:
iecv_auc_curve(X, y, log_model)

## CV per poli

In [ ]:
X, y = appointments_features.drop(columns="no_show"), appointments_features["no_show"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, shuffle=False
)
train_groups = X.index.get_level_values("pseudo_id")

In [ ]:
for poli in X.index.get_level_values("hoofdagenda").unique():
    X_tmp = X.loc[(slice(None), slice(None), poli), :]
    y_tmp = y.loc[(slice(None), slice(None), poli)]
    train_groups_tmp = X_tmp.index.get_level_values("pseudo_id")

    _, _ = cv_auc_curve(
        X_tmp,
        y_tmp,
        lgboost_model,
        # HistGradientBoostingClassifier(learning_rate=0.05, max_iter=300),
        StratifiedGroupKFold(n_splits=5),
        train_groups_tmp,
        title=poli,
    )

## Adding poli as feature

In [ ]:
appointments_features_agenda = appointments_features.reset_index()
appointments_features_agenda["hoofdagenda_cat"] = appointments_features_agenda[
    "hoofdagenda"
].astype("category")
appointments_features_agenda = appointments_features_agenda.set_index(
    ["pseudo_id", "start", "hoofdagenda"]
)
X, y = (
    appointments_features_agenda.drop(columns="no_show"),
    appointments_features_agenda["no_show"],
)

model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=300,
    categorical_features=["hour", "weekday", "hoofdagenda_cat"],
)
train_groups = X.index.get_level_values("pseudo_id")

In [ ]:
X.dtypes

In [ ]:
_, _ = cv_auc_curve(
    X,
    y,
    model,
    StratifiedGroupKFold(n_splits=5),
    train_groups,
)

## Relplots

In [ ]:
X, y = featuretable.drop(columns="no_show"), featuretable["no_show"]

model = log_model.fit(X, y)

In [ ]:
len(log_model[-1].coef_[0])

In [ ]:
import relplot as rp

y_pred_total = model.predict_proba(X)
print("calibration error:", rp.smECE(y_pred_total[:, 1], y))
fig, ax = rp.rel_diagram(y_pred_total[:, 1], y)
fig.show()

## Calculate permutation importance per feature

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    model,
    X,
    y,
    n_repeats=10,
    random_state=42,
)

In [ ]:
plt.rcdefaults()

feature_importance_df = pd.DataFrame(
    tuple(zip(X.columns, result["importances_mean"], strict=True))
)
feature_importance_df.columns = ["feature", "importance"]
feature_importance_df = feature_importance_df.sort_values("importance", ascending=False)
feature_importance_df.plot.barh(x="feature", y="importance")

In [ ]:
model = lgboost_model.fit(X, y)

In [ ]:
result = permutation_importance(
    model,
    X,
    y,
    n_repeats=10,
    random_state=42,
)

In [ ]:
feature_importance_df = pd.DataFrame(
    tuple(zip(X.columns, result["importances_mean"], strict=True))
)
feature_importance_df.columns = ["feature", "importance"]
feature_importance_df = feature_importance_df.sort_values("importance", ascending=False)
feature_importance_df.plot.barh(x="feature", y="importance")